In [1]:
#import web3 and define FUJI net as provider
from web3 import Web3
my_provider = Web3.HTTPProvider("https://api.avax-test.network/ext/bc/C/rpc")
w3 = Web3(my_provider)

In [2]:
#check the connection
connected = w3.isConnected()
connected

True

In [5]:
#create a new account
account = w3.eth.account.create()
account_1 = account.address

In [7]:
#dont forget to transfer funds using the faucet
# navigate to https://faucet.avax-test.network/ and put the address stored in account_1
# request 20 avax
account_1

'0x2a2A7A5B1335af0a596A01bf7a3397e544D9a6CE'

In [8]:
#check account balance
balance=w3.eth.getBalance(account_1)
w3.fromWei(balance,'ether')

Decimal('20')

In [9]:
#this is the account private key. It must remain private.
account.privateKey

HexBytes('0x1e5b4fb6455ef8bf86a9b2be07bb4f4972d9f5807e1a367290c0f1d1464d8879')

In [10]:
#encrypt the privatekey
keystore = account.encrypt('something')
keystore

{'address': '2a2a7a5b1335af0a596a01bf7a3397e544d9a6ce',
 'crypto': {'cipher': 'aes-128-ctr',
  'cipherparams': {'iv': '81aaad419e00a4be050767d10716d230'},
  'ciphertext': 'd397fcfb4c685b4d507955f6389ccabc16250e04e78951fa9d6ed938e6c1b583',
  'kdf': 'scrypt',
  'kdfparams': {'dklen': 32,
   'n': 262144,
   'r': 1,
   'p': 8,
   'salt': '57dda9d94e318dbc29222f4fefd917c1'},
  'mac': '8152a37611a19a19c56c3994721cfbacb86ac4da34db61b710e179b8c1adb626'},
 'id': 'ec3bd3a9-4176-4c94-80ce-ca2da3c0ae45',
 'version': 3}

In [12]:
#decrypt the keystore using the previous password
privatekey = w3.eth.account.decrypt(keystore, 'something')
privatekey

HexBytes('0x1e5b4fb6455ef8bf86a9b2be07bb4f4972d9f5807e1a367290c0f1d1464d8879')

In [16]:
#import solcx and check the current version
import solcx
from solcx import compile_standard
solcx.install_solc()

Version('0.8.4')

In [17]:
#compile a solidity smart contract
compiled_sol = compile_standard({
     "language": "Solidity",
     "sources": {
         "Storage.sol": {
             "content": '''   
// SPDX-License-Identifier: GPL-3.0
pragma solidity >=0.7.0 <0.8.5;

/**
 * @title Storage
 * @dev Store & retrieve value in a variable
 */
contract Storage {

    uint256 number;

    /**
     * @dev Store value in variable
     * @param num value to store
     */
    function store(uint256 num) public {
        number = num;
    }

    /**
     * @dev Return value 
     * @return value of 'number'
     */
    function retrieve() public view returns (uint256){
        return number;
    }
}
'''
         }
     },
     "settings":
         {
             "outputSelection": {
                 "*": {
                     "*": [
                         "metadata", "evm.bytecode"
                         , "evm.bytecode.sourceMap"
                     ]
                 }
             }
         }
})
compiled_sol

{'contracts': {'Storage.sol': {'Storage': {'evm': {'bytecode': {'generatedSources': [],
      'linkReferences': {},
      'object': '608060405234801561001057600080fd5b5061012f806100206000396000f3fe6080604052348015600f57600080fd5b506004361060325760003560e01c80632e64cec11460375780636057361d146051575b600080fd5b603d6069565b6040516048919060c2565b60405180910390f35b6067600480360381019060639190608f565b6072565b005b60008054905090565b8060008190555050565b60008135905060898160e5565b92915050565b60006020828403121560a057600080fd5b600060ac84828501607c565b91505092915050565b60bc8160db565b82525050565b600060208201905060d5600083018460b5565b92915050565b6000819050919050565b60ec8160db565b811460f657600080fd5b5056fea26469706673582212202656cfb391c38a6f7fce4e3dc942e650b88a193cf09e8c901cad5881bd968ea864736f6c63430008040033',
      'opcodes': 'PUSH1 0x80 PUSH1 0x40 MSTORE CALLVALUE DUP1 ISZERO PUSH2 0x10 JUMPI PUSH1 0x0 DUP1 REVERT JUMPDEST POP PUSH2 0x12F DUP1 PUSH2 0x20 PUSH1 0x0 CODECOPY PUSH1 0x0 RETURN INVALID P

In [18]:
#deploy the smart contract using the account we created before
# get bytecode
import json
bytecode = compiled_sol['contracts']['Storage.sol']['Storage']['evm']['bytecode']['object']

# get abi
abi = json.loads(compiled_sol['contracts']['Storage.sol']['Storage']['metadata'])['output']['abi']

Storage = w3.eth.contract(abi=abi, bytecode=bytecode)

# Submit the transaction that deploys the contract

nonce = w3.eth.getTransactionCount(account_1)

withdraw_txn = Storage.constructor().buildTransaction({
    'nonce' : nonce,
    'chainId': 43113,
    'gasPrice': w3.eth.gas_price,
    'gas': 1000000
})

signed_tx = w3.eth.account.signTransaction(withdraw_txn,privatekey)
tx_hash =w3.eth.send_raw_transaction(signed_tx.rawTransaction)
w3.eth.waitForTransactionReceipt(tx_hash)

#signed_tx = w3.eth.account.signTransaction(tx,privatekey)

#tx_hash = Storage.constructor().transact(tx)

# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
tx_receipt

AttributeDict({'blockHash': HexBytes('0x6d1f3ed6a2e4a5ad5ce0175987d8a3b8c55cab702ec1f1534b1f6b375a9019b6'),
 'blockNumber': 275805,
 'contractAddress': '0x969105D9c07d4a4eE4245Ce3e46fCAC15dc27026',
 'cumulativeGasUsed': 118819,
 'from': '0x2a2A7A5B1335af0a596A01bf7a3397e544D9a6CE',
 'gasUsed': 118819,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'status': 1,
 'to': None,
 'transactionHash': HexBytes('0x54c8a5b7e7a5aec970443c4aaf50d97b3ef883ea8ea7963de66ec2426f390709'),
 'transactionIn

In [21]:
#access to the contract instance
addr=tx_receipt['contractAddress']
contract = w3.eth.contract(address=addr, abi=abi)

In [22]:
contract.all_functions()

[<Function retrieve()>, <Function store(uint256)>]

In [30]:
#check the stored value in the contract
contract.functions.retrieve().call()

0

In [32]:
#set a new store value in the framework contract
nonce = w3.eth.getTransactionCount(account_1)
withdraw_txn = contract.functions.store(10).buildTransaction({
    'nonce' : nonce,
    'chainId': 43113,
    'gasPrice': w3.eth.gas_price,
    'gas': 100000
})

signed_tx = w3.eth.account.signTransaction(withdraw_txn,privatekey)
tx_hash =w3.eth.send_raw_transaction(signed_tx.rawTransaction)
w3.eth.waitForTransactionReceipt(tx_hash)
tx_hash

HexBytes('0xedb77d040ec016a8e83682f4c1f1d65f34a78f65c50df7be5a97e34a9f6ddc02')

In [33]:
#check the stored value in the contract
contract.functions.retrieve().call()

10